In [4]:
import glob, os, subprocess
from IPython.display import IFrame

## Precursors

To train a model, you first need to convert your sequences and targets into the input HDF5 format. Check out my tutorials for how to do that; they're linked from the [main page](../README.md).

For this tutorial, grab a small example HDF5 that I constructed here with 10% of the training sequences and only GM12878 targets for various DNase-seq, ChIP-seq, and CAGE experiments.

In [2]:
if len(glob.glob('data/heart_l131k/tfrecords/*.tfr')) == 0:
    subprocess.call('curl -o data/heart_l131k.tgz https://storage.googleapis.com/basenji_tutorial_data/heart_l131k.tgz', shell=True)
    subprocess.call('tar -xzvf data/heart_l131k.tgz', shell=True)

## Train

Next, you need to decide what sort of architecture to use. This grammar probably needs work; my goal was to enable hyperparameter searches to write the parameters to file so that I could run parallel training jobs to explore the hyperparameter space. I included an example set of parameters that will work well with this data in models/params_small.txt.

Then, run [basenji_train.py](https://github.com/calico/basenji/blob/master/bin/basenji_train.py) to train a model. The program will offer training feedback via stdout and write the model output files to the prefix given by the *-s* parameter.

The most relevant options here are:

| Option/Argument | Value | Note |
|:---|:---|:---|
| --augment_rc | True | Process even-numbered epochs as forward, odd-numbered as reverse complemented. |
| --ensemble_rc | True | Average forward and reverse complemented predictions on validation set. |
| --augment_shifts | "1,0,-1" | Rotate epochs over small sequence shifts. |
| --logdir | models/heart | Directory to save training logs and model checkpoints. |
| --params | models/params_small.txt | Table of parameters to setup the model architecture and optimization. |
| --train_data | data/heart_l131k/tfrecords/train*.tfr | Training set TFRecords. |
| --test_data | data/heart_l131k/tfrecords/valid*.tfr | Validation set TFRecords. |

If you want to train, uncomment the following line and run it. Depending on your hardware, it may require several hours.

In [2]:
! basenji_train.py --augment_rc --ensemble_rc --augment_shifts "1,0,-1" --logdir models/heart --params models/params_small.txt --train_data data/heart_l131k/tfrecords/train*.tfr --test_data data/heart_l131k/tfrecords/valid*.tfr

{'batch_size': 4, 'batch_buffer': 4096, 'link': 'softplus', 'loss': 'poisson', 'optimizer': 'adam', 'adam_beta1': 0.97, 'adam_beta2': 0.98, 'learning_rate': 0.002, 'num_targets': 3, 'target_pool': 128, 'seq_length': 131072, 'target_length': 1024, 'cnn_dropout': 0.1, 'cnn_filter_sizes': [20, 7, 7, 7, 3, 3, 3, 3, 3, 3, 3, 1], 'cnn_filters': [128, 128, 192, 256, 256, 32, 32, 32, 32, 32, 32, 384], 'cnn_pool': [2, 4, 4, 4, 1, 0, 0, 0, 0, 0, 0, 0], 'cnn_dilation': [1, 1, 1, 1, 1, 2, 4, 8, 16, 32, 64, 1], 'cnn_dense': [0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0]}
2019-05-22 16:32:53.151899: I tensorflow/core/platform/cpu_feature_guard.cc:141] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
data/heart_l131k/tfrecords/train-0.tfr has 256 sequences with 3/3 targets
Cannot order TFRecords data/heart_l131k/tfrecords/train-0.tfr
Instructions for updating:
Colocations handled automatically by placer.
data/heart_l131k/tfrecords/valid-0.tfr has 256 sequences with 3

## Test

Alternatively, you can just download a trained model.

In [5]:
if not os.path.isdir('models/heart'):
    os.mkdir('models/heart')
if not os.path.isfile('models/heart/model_best.tf.meta'):
    subprocess.call('curl -o models/heart/model_best.tf.index https://storage.googleapis.com/basenji_tutorial_data/model_best.tf.index', shell=True)
    subprocess.call('curl -o models/heart/model_best.tf.meta https://storage.googleapis.com/basenji_tutorial_data/model_best.tf.meta', shell=True)
    subprocess.call('curl -o models/heart/model_best.tf.data-00000-of-00001 https://storage.googleapis.com/basenji_tutorial_data/model_best.tf.data-00000-of-00001', shell=True)

models/heart/model_best.tf will now specify the name of your saved model to be provided to other programs.

To further benchmark the accuracy (e.g. computing significant "peak" accuracy), use [basenji_test.py](https://github.com/calico/basenji/blob/master/bin/basenji_test.py).

The most relevant options here are:

| Option/Argument | Value | Note |
|:---|:---|:---|
| --rc | | Average the forward and reverse complement to form prediction. |
| -o | output/heart_test | Output directory. |
| --ai | 0,1,2 | Make accuracy scatter plots for targets 0, 1, and 2. |
| --ti | 3,4,5 | Make BigWig tracks for targets 3, 4, and 5. |
| -t | data/heart_l131k.bed | BED file describing sequence regions for BigWig track output. |
| params_file | models/params_small.txt | Table of parameters to setup the model architecture and optimization. |
| model_file | models/heart/model_best.tf | Trained saved model prefix. |
| data_file | data/heart_l131k.h5 | HDF5 file containing the test input and output datasets as generated by [basenji_hdf5_single.py](https://github.com/calico/basenji/blob/master/bin/basenji_hdf5_single.py) |

In [6]:
! basenji_test.py --ai 0,1,2 -o output/heart_test --rc --shifts "1,0,-1" models/params_small.txt models/heart/model_best.tf data/heart_l131k

/Users/davidkelley/code/Basenji/bin/basenji_test.py:118: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  targets_df = pd.read_table(options.targets_file, index_col=0)
{'batch_size': 4, 'batch_buffer': 4096, 'link': 'softplus', 'loss': 'poisson', 'optimizer': 'adam', 'adam_beta1': 0.97, 'adam_beta2': 0.98, 'learning_rate': 0.002, 'num_targets': 3, 'target_pool': 128, 'seq_length': 131072, 'target_length': 1024, 'cnn_dropout': 0.1, 'cnn_filter_sizes': [20, 7, 7, 7, 3, 3, 3, 3, 3, 3, 3, 1], 'cnn_filters': [128, 128, 192, 256, 256, 32, 32, 32, 32, 32, 32, 384], 'cnn_pool': [2, 4, 4, 4, 1, 0, 0, 0, 0, 0, 0, 0], 'cnn_dilation': [1, 1, 1, 1, 1, 2, 4, 8, 16, 32, 64, 1], 'cnn_dense': [0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0]}
2019-05-22 17:13:50.662575: I tensorflow/core/platform/cpu_feature_guard.cc:141] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
data/heart_l131k/tfrecords/test-*.tfr has 566 sequences with 3/3 targ

*data/heart_test/acc.txt* is a table specifiying the loss function value, R2, R2 after log2, and Spearman correlation for each dataset. 

In [7]:
! cat output/heart_test/acc.txt

   0  0.31803  0.30800  0.56061  0.57796   CNhs11760  aorta
   1  0.18990  0.37857  0.61808  0.69475   CNhs12843  artery
   2  0.73150  0.26495  0.51856  0.51211   CNhs12856  pulmonic_valve


The directories *pr*, *roc*, *violin*, and *scatter* in *data/heart_test* contain plots for the targets indexed by 0, 1, and 2 as specified by the --ai option above.

E.g.

In [11]:
IFrame('output/heart_test/pr/t0.pdf', width=600, height=500)